In [1]:
import json
from datetime import date
import polars as pl
import numpy as np

In [2]:
schema={
    'id': pl.Utf8,
    'abstract': pl.Utf8,
    'beratungsstand': pl.Utf8,
    'vorgangstyp': pl.Utf8,
    'gesta': pl.Utf8,
    'archiv': pl.Utf8,
    'sachgebiet': pl.List(pl.Utf8),
    'typ': pl.Utf8,
    'wahlperiode': pl.UInt64,
    'zustimmungsbeduerftigkeit': pl.List(pl.Utf8),
    'initiative': pl.List(pl.Utf8),
    'aktualisiert': pl.Utf8,
    'titel': pl.Utf8,
    'datum': pl.Utf8,
    'verkuendung': pl.List(pl.Struct([
        pl.Field('verkuendungsdatum', pl.Utf8)
    ]))
}
df_vorgang = pl.DataFrame(json.load(open('vorgang.json', encoding='utf-8')), schema=schema)
df_gesetze = df_vorgang.filter(pl.col('vorgangstyp') == 'Gesetzgebung')
pl.Config(fmt_str_lengths=100)
df_gesetze['beratungsstand'].unique()

beratungsstand
str
"""Einbringung abgelehnt"""
"""Den Ausschüssen zugewiesen"""
"""Überwiesen"""
"""Verkündet"""
"""Angenommen"""
"""Für erledigt erklärt"""
"""Bundesrat hat zugestimmt"""
"""Dem Bundesrat zugeleitet - Noch nicht beraten"""
"""Abgelehnt"""


In [3]:
df_vorgangspos = pl.read_json('vorgangsposition.json')
df_vorgangspos = df_vorgangspos.with_columns(
   pl.col("datum").str.to_date(format="%Y-%m-%d")
)
df_vorgangspos = df_vorgangspos.filter((pl.col('vorgangstyp') == 'Gesetzgebung') & (pl.col('typ') == 'Vorgangsposition'))
df_vorgang_duration = df_vorgangspos.group_by(['vorgang_id']).agg([
    pl.col('datum').min().alias('vorgang_start'), 
])

In [4]:
df_gesetze = df_gesetze.join(df_vorgang_duration.select(['vorgang_id', 'vorgang_start']), left_on='id', right_on='vorgang_id', how='left')

In [5]:
today = date.today()
df_gesetze = df_gesetze.with_columns(
    pl.when(pl.col('beratungsstand') == 'Verkündet').then(
        pl.col('verkuendung').list.eval(pl.col('').struct.field('verkuendungsdatum').str.to_date(format="%Y-%m-%d")).list.min()
    ).when(pl.col('beratungsstand') == 'Abgelehnt').then(
        pl.col('datum').str.to_date(format="%Y-%m-%d")
    ).otherwise(today).alias('vorgang_end')
)
df_gesetze = df_gesetze.with_columns((pl.col('vorgang_end') - pl.col('vorgang_start')).dt.days().alias('vorgangsdauer'))

In [6]:
df_gesetze.write_json('gesetze.json', row_oriented=True)

In [7]:
df_gesetze_public = df_gesetze.filter(pl.col('beratungsstand') == 'Verkündet')
bins = [7, 14, 30, 60, 120, 240, 365]
duration_bin = np.digitize(df_gesetze_public.select('vorgangsdauer').to_series(), bins, right=True)
df_gesetze_public.select('vorgangsdauer').to_series().hist(bins)

break_point,category,vorgangsdauer_count
f64,cat,u32
7.0,"""(-inf, 7.0]""",1
14.0,"""(7.0, 14.0]""",3
30.0,"""(14.0, 30.0]""",17
60.0,"""(30.0, 60.0]""",20
120.0,"""(60.0, 120.0]""",52
240.0,"""(120.0, 240.0]""",66
365.0,"""(240.0, 365.0]""",3
inf,"""(365.0, inf]""",0


In [8]:
df_gesetze_public_bins = df_gesetze_public.with_columns(pl.from_numpy(duration_bin, schema=['duration_bin']))
df_gesetze_public_bins.filter(pl.col('duration_bin') == pl.col('duration_bin').min())

id,abstract,beratungsstand,vorgangstyp,gesta,archiv,sachgebiet,typ,wahlperiode,zustimmungsbeduerftigkeit,initiative,aktualisiert,titel,datum,verkuendung,vorgang_start,vorgang_end,vorgangsdauer,duration_bin
str,str,str,str,str,str,list[str],str,u64,list[str],list[str],str,str,str,list[struct[1]],date,date,i64,i64
"""283063""","""Impfpflicht gegen COVID-19 bzw. Nachweis über Genesung oder Kontraindikation für Beschäftigte besti…","""Verkündet""","""Gesetzgebung""","""M002""","""XX/2""","[""Gesundheit""]","""Vorgang""",20,"[""Ja, laut Gesetzentwurf (Drs 20/188)"", ""Ja, laut Verkündung (BGBl I)""]","[""Fraktion BÜNDNIS 90/DIE GRÜNEN"", ""Fraktion der FDP"", ""Fraktion der SPD""]","""2022-07-26T19:57:25+02:00""","""Gesetz zur Stärkung der Impfprävention gegen COVID-19 und zur Änderung weiterer Vorschriften im Zus…","""2021-12-10""","[{""2021-12-11""}]",2021-12-06,2021-12-11,5,0


In [9]:
df_gesetze_public_bins.filter(pl.col('duration_bin') == pl.col('duration_bin').max())

id,abstract,beratungsstand,vorgangstyp,gesta,archiv,sachgebiet,typ,wahlperiode,zustimmungsbeduerftigkeit,initiative,aktualisiert,titel,datum,verkuendung,vorgang_start,vorgang_end,vorgangsdauer,duration_bin
str,str,str,str,str,str,list[str],str,u64,list[str],list[str],str,str,str,list[struct[1]],date,date,i64,i64
"""292499""","""Schrittweise Einführung einer Kennzeichnungspflicht für Lebensmittel tierischen Ursprungs, beginnen…","""Verkündet""","""Gesetzgebung""","""F008""",null,"[""Landwirtschaft und Ernährung""]","""Vorgang""",20,"[""Nein, laut Gesetzentwurf (Drs 505/22)"", ""Nein, laut Verkündung (BGBl I)""]","[""Bundesregierung""]","""2023-08-24T16:05:21+02:00""","""Gesetz zur Kennzeichnung von Lebensmitteln mit der Haltungsform der Tiere, von denen die Lebensmitt…","""2023-07-07""","[{""2023-08-23""}]",2022-10-14,2023-08-23,313,6
"""290260""","""Umsetzung der sog. EU-Whistleblower-Richtlinie für einen verbesserten Hinweisgeberschutz und zur Sc…","""Verkündet""","""Gesetzgebung""","""C034""","""XX/136""","[""Arbeit und Beschäftigung"", ""Recht"", … ""Wirtschaft""]","""Vorgang""",20,"[""Ja, laut Gesetzentwurf (Drs 372/22)"", ""Ja, laut Verkündung (BGBl I)""]","[""Bundesregierung""]","""2023-06-14T12:20:41+02:00""","""Gesetz für einen besseren Schutz hinweisgebender Personen sowie zur Umsetzung der Richtlinie zum Sc…","""2023-05-12""","[{""2023-06-02""}]",2022-08-05,2023-06-02,301,6
"""287336""","""Zur Stärkung der Mitgliedschaftsrechte und Förderung ehrenamtlichen Engagements Verstetigung pandem…","""Verkündet""","""Gesetzgebung""","""C025""","""XX/125""","[""Gesellschaftspolitik, soziale Gruppen"", ""Medien, Kommunikation und Informationstechnik"", ""Recht""]","""Vorgang""",20,"[""Nein, laut Gesetzesantrag (Drs 193/22)"", ""Nein, laut Verkündung (BGBl I)""]","[""Bayern""]","""2023-03-31T12:30:36+02:00""","""Gesetz zur Ermöglichung hybrider und virtueller Mitgliederversammlungen im Vereinsrecht""","""2023-03-03""","[{""2023-03-20""}]",2022-05-04,2023-03-20,320,6
